# Clasificación de datos posterior al agrupamiento

In [8]:
# utilizar un arbol de decisión para determinar las variables más importantes para el clustering y la precisión del modelo
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#Preprocesamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
#Data split y seleccion de modelos
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
# importar pipeline gridsearch

#Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

path = '../Data/dataset_clusterizado.xlsx'

dataset = pd.read_excel(path)

X = dataset.drop(columns=['cluster_kmeans', 'cluster_hc', 'Comunidad'])
y = dataset['cluster_kmeans']
display(X.head())
display(y.head())


,carrera,Espacio_Académico,alumnos,1p_total_actividades_asignadas,1p_total_actividades_evaluativas,1p_porcentaje_evidencias_calificadas,1p_total_actividades_calificadas,1p_total_actividades_entregadas,1p_porcentaje_evidencias_calificadas_tiempo,1p_total_actividades_calificadas_tiempo,...,2p_porcentaje_evidencias_calificadas,2p_total_actividades_calificadas,2p_total_actividades_entregadas,2p_porcentaje_evidencias_calificadas_tiempo,2p_total_actividades_calificadas_tiempo,2p_porcentaje_aprobados,total_actividades_calificadas,trabajos_por_calificar,correos_enviados,correos_recibidos
0,Ingeniería en Computación,Valle de Chalco,25,18,18,1.0,438,438.0,0.998,437.0,...,1.0,452.0,452.0,1.0,452.0,0,890.0,0.0,5.75,2.25
1,Ingeniería en Computación,Valle de Chalco,23,19,19,1.0,402,402.0,1.000,402.0,...,1.0,380.0,380.0,1.0,380.0,0,782.0,0.0,43.00,17.00
2,Ingeniería en Computación (2019),CU Texcoco,43,5,5,1.0,131,131.0,1.000,131.0,...,0.0,0.0,0.0,0.0,0.0,0,131.0,0.0,7.50,2.50
3,Ingeniería en Computación (2019),CU Texcoco,16,6,6,1.0,89,89.0,1.000,89.0,...,1.0,139.0,139.0,1.0,139.0,81,228.0,0.0,21.00,18.00
4,Ingeniería en Computación (2019),CU Texcoco,20,6,6,1.0,102,102.0,1.000,102.0,...,1.0,137.0,137.0,1.0,137.0,50,239.0,0.0,20.75,13.25


0    3
1    3
2    2
3    1
4    1
Name: cluster_kmeans, dtype: int64

In [15]:
# Ver balance de clases

print(y.value_counts())

cluster_kmeans
1    687
3    196
2    158
Name: count, dtype: int64


In [9]:
# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Identificar variables categóricas y numéricas
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [10]:
# Crear un pipeline para preprocesar los datos
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar los preprocesadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

# Definir los hiperparámetros a buscar
param_grid = {
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__splitter': ['best', 'random'],
    'classifier__max_depth': [None, 5, 10, 15, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [ 1, 2, 4,8]
}

# Definir el esquema de validación cruzada (StratifiedKFold)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Realizar la búsqueda en cuadrícula con validación cruzada en el pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='accuracy')

# Ajustar los datos usando GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['alumnos', '1p_total_actividades_asignadas',
       '1p_total_actividades_evaluativas',
       '1p_porcentaje_evidencias_calificadas',
       '1p_total_actividades_cali...
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['carrera', 'Espacio_Académico'], dtype='object'))])),
                                       ('classifier',
                                        DecisionTreeClassifier())]),
             param_grid={'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_depth': [None, 5, 10, 15, 20],
                         'classifier__min_samples_leaf': [1, 2, 4, 8],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__splitter': ['best', 'random']},
             scoring='accuracy')

In [11]:
# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Ajustar el mejor modelo en los datos de entrenamiento
best_model.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evalauamos en el train set solo para ver que no quedó sobreajustado
train_accuracy = best_model.score(X_train, y_train)
print("Train Accuracy:", train_accuracy)

# Evaluamos en el test set para ver como generaliza
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Best Hyperparameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__splitter': 'random'}
Train Accuracy: 0.9807692307692307
Test Accuracy: 0.9424920127795527


In [14]:
# Obtenemos las variables más importantes
importances = best_model.named_steps['classifier'].feature_importances_

# Ordenamos las importancias de mayor a menor
indices = np.argsort(importances)[::-1]

# Obtenemos los nombres de las columnas
columnas = X_train.columns
display(columnas)
# Mostramos las 10 variables más importantes
for i in range(5):
    print(f"{columnas[indices[i]]}: {importances[indices[i]]}")

Index(['carrera', 'Espacio_Académico', 'alumnos',
       '1p_total_actividades_asignadas', '1p_total_actividades_evaluativas',
       '1p_porcentaje_evidencias_calificadas',
       '1p_total_actividades_calificadas', '1p_total_actividades_entregadas',
       '1p_porcentaje_evidencias_calificadas_tiempo',
       '1p_total_actividades_calificadas_tiempo', '1p_porcentaje_aprobados',
       '2p_total_actividades_asignadas', '2p_total_actividades_evaluativas',
       '2p_porcentaje_evidencias_calificadas',
       '2p_total_actividades_calificadas', '2p_total_actividades_entregadas',
       '2p_porcentaje_evidencias_calificadas_tiempo',
       '2p_total_actividades_calificadas_tiempo', '2p_porcentaje_aprobados',
       'total_actividades_calificadas', 'trabajos_por_calificar',
       'correos_enviados', 'correos_recibidos'],
      dtype='object')

2p_total_actividades_calificadas_tiempo: 0.3662626113183254
2p_total_actividades_asignadas: 0.32862874456447616
2p_porcentaje_evidencias_calificadas_tiempo: 0.05517804820340307
1p_total_actividades_evaluativas: 0.03659808177159135
1p_total_actividades_entregadas: 0.03241916445886644
